In [ ]:
# Implement the basic PSO algorithm.

# PSO:
# Global topology (everyone knows about gbest)
# c1 = c2 = 1.49618
# linear decrease of inertia weight w from 0.9 to 0.4 with iterations
# Other settings are up to you (population/swarm size, number of iterations).

# ! Important notes !

# The equation for velocity computation is a vector equation.
# The initial velocity v0 is 0.
# Velocity is limited to a max of 20% of the search space range in each dimension.
# Tasks

# 1) Implement PSO
# 2) Test PSO on the same test functions

In [ ]:
# Import Libraries
import numpy as np

In [ ]:
# Define test functions
# Sphere function
def sphere(x):
    return np.sum(x**2)

# Schwefel function
def schwefel(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

# Rastrigin function
def rastrigin(x):
    return 10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

In [ ]:
# Define PSO algorithm
def pso(pop_size, dim, bounds, max_iter, obj_func, topology='global', c1=1.49618, c2=1.49618, w_range=(0.9, 0.4)):
    # Initialization
    swarm = np.random.uniform(bounds[0], bounds[1], size=(pop_size, dim))
    velocity = np.zeros_like(swarm)
    pbest = swarm.copy()
    pbest_values = np.array([obj_func(p) for p in pbest])

    if topology == 'global':
        gbest = pbest[np.argmin(pbest_values)]
        gbest_value = np.min(pbest_values)
    elif topology == 'ring':
        gbest = pbest[np.argmin(pbest_values)]
        gbest_value = np.min(pbest_values)
        # Define ring topology
        neighbors = {i: [(i - j) % pop_size for j in range(1, pop_size // 2 + 1)] + [(i + j) % pop_size for j in range(1, pop_size // 2 + 1)] for i in range(pop_size)}

    # Main loop
    for iteration in range(max_iter):
        w = w_range[0] - (w_range[0] - w_range[1]) * iteration / max_iter  # Linearly decrease inertia weight

        for i in range(pop_size):
            # Velocity update
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            if topology == 'global':
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - swarm[i]) + c2 * r2 * (gbest - swarm[i])
            elif topology == 'ring':
                ring_neighbors = neighbors[i]
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - swarm[i]) + c2 * r2 * (pbest[ring_neighbors].mean(axis=0) - swarm[i])

            # Velocity clipping
            v_max = 0.2 * (bounds[1] - bounds[0])
            velocity[i] = np.clip(velocity[i], -v_max, v_max)

            # Position update
            swarm[i] += velocity[i]

            # Position clipping
            swarm[i] = np.clip(swarm[i], bounds[0], bounds[1])

            # Update personal best
            if obj_func(swarm[i]) < pbest_values[i]:
                pbest[i] = swarm[i]
                pbest_values[i] = obj_func(swarm[i])

        # Update global best
        if topology == 'global':
            gbest_index = np.argmin(pbest_values)
            if pbest_values[gbest_index] < gbest_value:
                gbest = pbest[gbest_index]
                gbest_value = pbest_values[gbest_index]
        elif topology == 'ring':
            gbest_index = np.argmin(pbest_values)
            if pbest_values[gbest_index] < gbest_value:
                gbest = pbest[gbest_index]
                gbest_value = pbest_values[gbest_index]

    return gbest, gbest_value



In [ ]:
# Test PSO on Sphere, Schwefel, and Rastrigin functions
pop_size = 50
dim = 10
max_iter = 100

In [ ]:
# Define bounds for each function
bounds = {
    'sphere': (-5.12, 5.12),
    'schwefel': (-500, 500),
    'rastrigin': (-5.12, 5.12)
}

In [ ]:
# Test PSO with Global Topology
print("PSO - Global Topology:")
for obj_func, func_bounds in zip([sphere, schwefel, rastrigin], bounds.values()):
    best_solution, best_value = pso(pop_size, dim, func_bounds, max_iter, obj_func, topology='global', c1=1.49618, c2=1.49618, w_range=(0.9, 0.4))
    print(f"{obj_func.__name__}: Best Value = {best_value}, Best Solution = {best_solution}")

# Test PSO with Ring Topology
print("\nPSO - Ring Topology:")
for obj_func, func_bounds in zip([sphere, schwefel, rastrigin], bounds.values()):
    best_solution, best_value = pso(pop_size, dim, func_bounds, max_iter, obj_func, topology='ring', c1=1.49618, c2=1.49618, w_range=(0.9, 0.4))
    print(f"{obj_func.__name__}: Best Value = {best_value}, Best Solution = {best_solution}")

PSO - Global Topology:
sphere: Best Value = 8.552420457885311e-09, Best Solution = [ 2.08652023e-05  4.28758796e-05  1.36841115e-05  1.02389706e-06
 -2.08838026e-05  1.51284882e-05  3.27585208e-05 -2.56984576e-05
  4.74085018e-05  3.80041792e-05]
schwefel: Best Value = 1085.7014601361202, Best Solution = [-124.81234243  203.83400848  203.81809822  420.96890143 -302.50819107
 -302.51264716  420.955743   -302.505452    420.95676805  420.98770914]
rastrigin: Best Value = 10.945251740679538, Best Solution = [ 9.95219901e-01  9.94071316e-01 -9.95382262e-01 -9.93982683e-01
  9.94748488e-01 -5.84419111e-04  9.94128474e-01  2.33828361e-04
 -9.94611590e-01  1.98933855e+00]

PSO - Ring Topology:
sphere: Best Value = 0.00010677461783000019, Best Solution = [-0.00456869 -0.00078018  0.00240663  0.00088413 -0.00150111  0.00190369
 -0.00153963  0.00072413  0.00404875 -0.00731812]
schwefel: Best Value = 2236.676413935019, Best Solution = [-278.15222783 -203.3627237   100.68464245 -295.3072221   408.4